In [2]:
!mkdir -p ~/.kaggle
!cp kaggle(1).json ~/.kaggle/

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `cp kaggle(1).json ~/.kaggle/'


In [3]:
!kaggle datasets download -d bittlingmayer/amazonreviews

Dataset URL: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews
License(s): unknown
amazonreviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/amazonreviews.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/amazonreviews.zip' mode='r'>>

In [5]:
import pandas as pd

# Specify the separator and handle potential bad lines
train_df = pd.read_csv("/content/train.ft.txt.bz2", header=None, names=['text', 'sentiment'], sep='\t', on_bad_lines='warn')

In [6]:
df = train_df.copy()

In [7]:
train_df = train_df.sample(20000)

In [8]:
import re

In [9]:
def extract_label_and_clean_text(text):
    match = re.match(r'__label__(\d+)\s+(.*)', text)
    if match:
        label = int(match.group(1))
        clean_text = match.group(2)
        return label, clean_text
    return None, text

# Apply the function to the DataFrame
train_df['sentiment'], train_df['clean_text'] = zip(*train_df['text'].apply(extract_label_and_clean_text))

# Display the DataFrame
print(train_df[['sentiment', 'clean_text']])

         sentiment                                         clean_text
620131           1  Great while it lasted: I thought we finally ha...
3347656          1  Does not fit SIG 229 40 S&W: The holster appea...
2479415          2  Gritty,heartrending, and imaginative: Great st...
2356209          1  Doesn't work properly: After a long wait for t...
1370234          2  The Best CD('s) Can Buy.... Or Can It?: ... Ou...
...            ...                                                ...
3057190          2  WOW: i love this! im only 14 so i dont really ...
2701290          2  Song with this book better than on "No": We li...
1542912          1  Ok!?: I thought the shark MX was ok. Its keybo...
1308793          2  Great product.: Very small, i hardly feel it. ...
3066266          2  Very Moving, Beautiful Tales of Snow and a God...

[20000 rows x 2 columns]


In [10]:
train_df.drop(columns=["text"], inplace = True)

In [11]:
df = train_df.copy()

In [12]:
data =df

In [13]:
data["label"] = data["sentiment"].apply(lambda x: "positive" if x == 2 else "negative")

In [14]:
data.rename(columns={"clean_text": "Review"}, inplace=True)

In [15]:
!pip install emoji
import emoji
import re

def text_cleaning(text):
    text = emoji.demojize(text)
    text = re.sub('n\'t', 'not', text)
    return text
data['Review'] = data['Review'].apply(text_cleaning)
data = data.drop(['sentiment'], axis=1)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_valid_test, y_train, y_valid_test = train_test_split(data.loc[:, data.columns!='label'],
                                                                data['label'],
                                                                test_size=0.2,
                                                                stratify=data['label'])
X_valid, X_test, y_valid, y_test = train_test_split(X_valid_test,
                                                    y_valid_test,
                                                    test_size=0.5,
                                                    stratify=y_valid_test)

In [17]:
data_train = pd.DataFrame([X_train['Review'], y_train]).T
data_test = pd.DataFrame([X_test['Review'], y_test]).T
data_valid = pd.DataFrame([X_valid['Review'], y_valid]).T

In [18]:
!pip install datasets
from datasets import DatasetDict, Dataset
dts = DatasetDict()
dts['train'] = Dataset.from_pandas(data_train)
dts['test'] = Dataset.from_pandas(data_test)
dts['valid'] = Dataset.from_pandas(data_valid)

In [19]:
import numpy as np

In [22]:
import pandas as pd
import numpy as np
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset, DatasetDict
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


label2id = {'negative': 0, 'positive': 1}
id2label = {value: key for key, value in label2id.items()}

model_ckpt = 'google-bert/bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                           id2label=id2label,
                                                           label2id=label2id,
                                                           num_labels=2)

def tokenize(examples):
    tokens = tokenizer(examples['Review'],
                       max_length=512,
                       truncation=True,
                       padding='max_length'
                      )
    # Ensure labels are integers
    tokens['label'] = [label2id[l] for l in examples['label']]  # Convert labels to integers
    return tokens

# Tokenize the dataset
dts = dts.map(tokenize, batched=True, batch_size=64)

# Define the compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = np.argmax(eval_pred.predictions, axis=-1), eval_pred.label_ids
    return {
        'precision': precision_score(labels, predictions, average='weighted'),
        'recall': recall_score(labels, predictions, average='weighted'),
        'f1': f1_score(labels, predictions, average='weighted'),
        'accuracy': accuracy_score(labels, predictions)
    }

# Training arguments
training_arguments = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=dts['train'],
    eval_dataset=dts['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.245800,0.212690,0.933018,0.933000,0.933001,0.933000
2,0.097500,0.264303,0.942003,0.942000,0.942000,0.942000


TrainOutput(global_step=4000, training_loss=0.20571357154846193, metrics={'train_runtime': 3462.0098, 'train_samples_per_second': 9.243, 'train_steps_per_second': 1.155, 'total_flos': 8419553771520000.0, 'train_loss': 0.20571357154846193, 'epoch': 2.0})

In [23]:
model.save_pretrained('saved_model')
tokenizer.save_pretrained('saved_model')

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [24]:
import shutil

shutil.make_archive('saved_model', 'zip', 'saved_model')

'/content/saved_model.zip'